# Example: Using DARTS Deep Models via TSFB

Notebook này minh họa cách làm việc với DARTS_DEEP_MODELS trong dự án tsfb, giống như khi dùng một thư viện dự báo riêng biệt.
Ở đây sử dụng mô hình học sâu BlockRNNModel của thư viện darts, được TSFB bao bọc lại.

Layout gồm có:
1. Tạo một bộ dữ liệu chuỗi thời gian đơn biến đơn giản.

2. Định nghĩa các siêu tham số (hyperparameters) cần thiết cho mô hình DARTS.

3. Khởi tạo mô hình thông qua adapter của TSFB (get_models).

4. Huấn luyện mô hình với dữ liệu và tạo dự báo.

In [1]:
import pandas as pd
import numpy as np
#Step 1: Tạo data 
# Tạo date index 
index = pd.date_range(start='2020-01-01', periods=200, freq='D')

# Tạo synthetic sine wave với Gaussian noise
signal = np.sin(2 * np.pi * np.arange(200) / 30)
noise = np.random.normal(scale=0.1, size=200)
y_values = signal + noise  # univariate target series

# Tạo dataframe
data = pd.DataFrame({'y': y_values}, index=index)

data.head()

,y
2020-01-01,0.113417
2020-01-02,0.212026
2020-01-03,0.330033
2020-01-04,0.621091
2020-01-05,0.533333


In [ ]:
# Step 2: định nghĩa các hyperparameters cho model
from tsfb.conf import darts_conf

# Optional: Ghi đè mặc định toàn cục cho Apple Silicon: dùng CPU
darts_conf.DEEP_MODEL_ARGS['pl_trainer_kwargs'].update({
    'accelerator': 'cpu',  
    'devices': 1,
})

# Tạo config giả
fake_config = {
    "models": [
        {
            "model_name": "darts.models.BlockRNNModel",
            "model_hyper_params": {
                "input_chunk_length": 24,
                "output_chunk_length": 7
            }
        }
    ]
}

In [25]:
# Step 3: Khởi tạo model
from tsfb.base.models.model_loader import get_models

# Lấy danh sách ModelFactory dựa trên cấu hình
model_factories = get_models(fake_config)

# Khởi tạo mô hình đầu tiên (vì đang chỉ khai báo một mô hình)
model = model_factories[0]()

model

BlockRNNModel

In [27]:
# Step 4: fit model, produce a forecast
# Fit model
model.forecast_fit(train_data=data, train_ratio_in_tv=0.9)

# Dùng forecast() để dự báo cho các 'horizon' kế tiếp
predictions = model.forecast(horizon=7, series=data)

# Tạo DataFrame cho dữ liệu dự báo
future_index = pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=7, freq='D')
preds_df = pd.DataFrame(predictions, index=future_index, columns=['prediction'])

preds_df

/opt/miniconda3/envs/vnpt_ts311_arm/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/opt/miniconda3/envs/vnpt_ts311_arm/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/miniconda3/envs/vnpt_ts311_arm/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/opt/miniconda3/envs/vnpt_ts311_arm/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


,prediction
2020-07-19,-0.877529
2020-07-20,-0.943350
2020-07-21,-1.011931
2020-07-22,-0.976214
2020-07-23,-0.946915
2020-07-24,-0.855430
2020-07-25,-0.753514
